# Google Colab Rust Setup

The following cell is used to set up and spin up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 

In [ ]:
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Working with Devices I/O

### Opening a File

In [2]:
use std::fs::File;

fn main() -> Result<(), std::io::Error> {
    let file = File::open("example.txt")?;
    // Now, 'file' is a handle to the opened file.
    Ok(())
}

#### Reading from a File

In [4]:
use std::fs::File;
use std::io::Read;

fn main() -> Result<(), std::io::Error> {
    let mut file = File::open("example.txt")?;
    let mut contents = String::new();
    file.read_to_string(&mut contents)?;
    // 'contents' now holds the content of the file.
    Ok(())
}

main()

Ok(())

In [5]:
use std::fs::File;
use std::io::{BufRead, BufReader};

fn main() -> Result<(), std::io::Error> {
    let file = File::open("example.txt")?;
    let reader = BufReader::new(file);

    for line in reader.lines() {
        let line = line?;
        // Process each line as needed.
    }

    Ok(())
}

main()

Ok(())

#### Writing to a File

In [6]:
use std::fs::File;
use std::io::Write;

fn main() -> Result<(), std::io::Error> {
    let mut file = File::create("new_file.txt")?;
    file.write_all(b"Hello, Rust!")?;
    Ok(())
}

main()

Ok(())

In [7]:
:dep serde = {version = "1.0.193", features = ["derive"]}

In [10]:
:dep serde_json = {version = "1.0.108"}

In [11]:
use serde::{Serialize, Deserialize};
use std::fs::File;
use serde_json::to_writer;


#[derive(Serialize, Deserialize)]
struct Configuration {
    name: String,
    value: i32,
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let config = Configuration {
        name: String::from("setting"),
        value: 42,
    };

    let file = File::create("config.json")?;
    to_writer(file, &config)?;

    Ok(())
}

main()

Ok(())

#### Closing a File

In [12]:
use std::fs::File;
use std::io::Write;

fn main() -> Result<(), std::io::Error> {
    let mut file = File::create("new_file.txt")?;
    file.write_all(b"Hello, Rust!")?;
    // The 'file' variable goes out of scope here, and the file is automatically closed.
    Ok(())
}

main()

Ok(())

#### Creating Directories

In [13]:
use std::fs;

fn main() -> Result<(), std::io::Error> {
    fs::create_dir("new_directory")?;
    Ok(())
}

main()

Ok(())

#### Renaming and Moving Files

In [14]:
use std::fs;

fn main() -> Result<(), std::io::Error> {
    fs::rename("old_file.txt", "new_file.txt")?;
    Ok(())
}

main()

Ok(())

#### Checking File Metadata

In [15]:
:dep chrono = {version = "0.4.31"}

In [16]:
use chrono::NaiveDateTime;
use std::fs::metadata;
use std::io;
use std::os::unix::fs::PermissionsExt;
use std::time::SystemTime;

fn main() -> io::Result<()> {
    let metadata = metadata("example.txt")?;
    println!("File size: {} bytes", metadata.len());

    if let Ok(modified_time) = metadata.modified() {
        let modified_time = modified_time
            .duration_since(SystemTime::UNIX_EPOCH)
            .unwrap_or_default();

        let modified_datetime =
            NaiveDateTime::from_timestamp_opt(modified_time.as_secs() as i64, 0).unwrap();

        println!("Last modified: {}", modified_datetime);
    } else {
        eprintln!("Unable to retrieve modified time");
    }
    let permissions = metadata.permissions().mode();
    println!("Permissions: {:o}", permissions & 0o777);

    Ok(())
}

main()

File size: 0 bytes
Last modified: 2023-11-01 19:24:56
Permissions: 755


Ok(())

#### Deleting Files and Directories

In [17]:
use std::fs;

fn main() -> Result<(), std::io::Error> {
    fs::remove_file("file_to_delete.txt")?;
    fs::remove_dir("directory_to_delete")?;
    Ok(())
}

main()

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

#### Traversing Directories

In [20]:
use std::fs;
use std::os::unix::fs::FileTypeExt;
use std::path::Path;

fn main() -> Result<(), std::io::Error> {
    let dir = fs::read_dir("path_to_directory")?;

    for entry in dir {
        let entry = entry?;
        let path = entry.path();
        println!("Entry name: {:?}", path.file_name().unwrap_or_default());

        let file_type = entry.file_type()?;
        let file_type_str = if file_type.is_file() {
            "File"
        } else if file_type.is_dir() {
            "Directory"
        } else if file_type.is_symlink() {
            "Symbolic Link"
        } else if file_type.is_char_device() {
            "Character Device"
        } else if file_type.is_block_device() {
            "Block Device"
        } else if file_type.is_fifo() {
            "FIFO (Named Pipe)"
        } else if file_type.is_socket() {
            "Socket"
        } else {
            "Unknown"
        };

        println!("Entry type: {}", file_type_str);
    }

    Ok(())
}

main()

Entry name: "config.json"
Entry type: File
Entry name: "new_file.txt"
Entry type: File


Ok(())

#### Working with File Paths

In [21]:
use std::path::PathBuf;

fn main() {
    let base_path = PathBuf::from("/usr/local");
    let new_path = base_path.join("bin");

    println!("New path: {:?}", new_path);
}

main()

New path: "/usr/local/bin"


()

In [22]:
use std::path::Path;

fn main() {
    let path = Path::new("/usr/local/bin");
    if let Some(parent) = path.parent() {
        println!("Parent directory: {:?}", parent);
    }
}

main()

Parent directory: "/usr/local"


()

#### Reading and Writing Binary Data

In [23]:
use std::fs::File;
use std::io::Read;

fn main() -> Result<(), std::io::Error> {
    let mut file = File::open("binary_file.bin")?;
    let mut buffer = Vec::new();

    file.read_to_end(&mut buffer)?;

    // 'buffer' now contains the binary data from the file.

    Ok(())
}

main()

Ok(())

In [24]:
use std::fs::File;
use std::io::Write;

fn main() -> Result<(), std::io::Error> {
    let mut file = File::create("binary_output.bin")?;
    let data = vec![0x48, 0x65, 0x6C, 0x6C, 0x6F];
    file.write_all(&data)?;

    // The binary data has been successfully written to the file.

    Ok(())
}

main()

Ok(())

#### Error Handling and Result Types

In [26]:
use std::fs::File;
use std::io::{Read, Write};

fn read_and_write_files() -> Result<(), std::io::Error> {
    let mut read_file = File::open("input.txt")?;

    let mut write_file = File::create("output.txt")?;

    let mut buffer = Vec::new();

    read_file.read_to_end(&mut buffer)?;

    write_file.write_all(&buffer)?;

    Ok(())
}

fn main() {
    match read_and_write_files() {
        Ok(_) => println!("File operations completed successfully."),
        Err(e) => eprintln!("Error: {}", e),
    }
}

main()

File operations completed successfully.


()

#### Overview of Working with Hardware Devices

In [27]:
:dep rppal = {version = "0.15.0"}

In [29]:
use rppal::gpio::Gpio;
use std::error::Error;

fn main() -> Result<(), Box<dyn Error>> {
    let gpio = Gpio::new()?;
    let pin = gpio.get(17)?;

    let mut pin = pin.into_output();

    // Handle any errors that may occur during pin operations
    if pin.set_high() != () {
        eprintln!("Error setting pin high");
    }

    // Do some work here, interacting with the GPIO pin

    if pin.set_low() != () {
        eprintln!("Error setting pin low");
    }

    // The pin is automatically released when it goes out of scope
    Ok(())
}

main()

Err(UnknownModel)

In [30]:
:dep i2cdev = {version = "0.6.0"}

In [32]:
use i2cdev::linux::LinuxI2CDevice;
use i2cdev::core::I2CDevice;
use std::error::Error;

fn main() -> Result<(), Box<dyn Error>> {
    let mut bus = LinuxI2CDevice::new("/dev/i2c-1", 0x68)?;

    let mut buffer: [u8; 2] = [0; 2];
    bus.write(&[0x1])?; // Command to read from the sensor
    bus.read(&mut buffer)?;

    let sensor_value = (u16::from(buffer[0]) << 8) | u16::from(buffer[1]);
    println!("Sensor value: {}", sensor_value);

    Ok(())
}

main()

Err(Io(Os { code: 13, kind: PermissionDenied, message: "Permission denied" }))

### Advanced File Operations

#### Symbolic Links and Hard Links

In [33]:
use std::os::unix::fs;

fs::symlink("target_file.txt", "symlink_to_target.txt")?;

In [35]:
use std::fs;

fs::hard_link("source_file.txt", "hard_link_to_source.txt")?;

#### Locking Files

In [36]:
:dep fs2 = {version = "0.4.3"}

In [37]:
use fs2::FileExt;
use std::fs::File;
use std::io;

fn main() -> Result<(), io::Error> {
    let file = File::create("locked_file.txt")?;

    // Attempt to acquire an exclusive lock on the file.
    if let Ok(mut _lock) = file.try_lock_exclusive() {
        println!("Acquired exclusive lock.");
        // Perform exclusive operations.

        // Lock is automatically released when 'lock' goes out of scope.
    } else {
        println!("Failed to acquire exclusive lock.");
    }

    Ok(())
}

main()

Acquired exclusive lock.


Ok(())

In [38]:
use fs2::FileExt;
use std::fs::{self, File};
use std::io;
use std::io::prelude::*;
use std::sync::Arc;
use std::thread;

fn main() -> Result<(), io::Error> {
    // Create or overwrite the file with some content.
    let content = "\nline 0\nline 1\nline 2\n";
    fs::write("shared_file.txt", content)?;

    // Read the file content outside the locked section.
    let file = File::open("shared_file.txt")?;
    let mut buffer = String::new();
    if let Ok(reader) = io::BufReader::new(file.try_clone()?).read_to_string(&mut buffer) {
        println!("Read {} bytes: {}", reader, &buffer);
    } else {
        println!("Failed to read the file.");
    }

    // Wrap the buffer in an Arc to share ownership.
    let buffer = Arc::new(buffer);

    // Spawn multiple threads to access the shared content.
    let num_threads = 5;
    let mut handles = vec![];

    for _ in 0..num_threads {
        let file_clone = file.try_clone();
        let buffer_clone = Arc::clone(&buffer); // Use Arc to share ownership

        let handle = thread::spawn(move || {
            if let Ok(mut _lock) = file_clone.unwrap().try_lock_shared() {
                println!(
                    "Thread {:?} acquired shared lock for read-only access.",
                    thread::current().id()
                );

                // Perform read-only operations on the shared content.
                println!(
                    "Thread {:?} is processing the shared content: {}",
                    thread::current().id(),
                    &buffer_clone // Use the shared Arc
                );
            } else {
                println!(
                    "Thread {:?} failed to acquire shared lock.",
                    thread::current().id()
                );
            }
        });

        handles.push(handle);
    }

    for handle in handles {
        handle.join().unwrap();
    }

    Ok(())
}

main()

Read 22 bytes: 
line 0
line 1
line 2

Thread ThreadId(2) acquired shared lock for read-only access.
Thread ThreadId(2) is processing the shared content: 
line 0
line 1
line 2

Thread ThreadId(1) acquired shared lock for read-only access.
Thread ThreadId(1) is processing the shared content: 
line 0
line 1
line 2

Thread ThreadId(4) acquired shared lock for read-only access.
Thread ThreadId(4) is processing the shared content: 
line 0
line 1
line 2

Thread ThreadId(5) acquired shared lock for read-only access.
Thread ThreadId(5) is processing the shared content: 
line 0
line 1
line 2

Thread ThreadId(3) acquired shared lock for read-only access.
Thread ThreadId(3) is processing the shared content: 
line 0
line 1
line 2



Ok(())

#### Memory-Mapped Files

In [39]:
:dep memmap = {version = "0.7.0"}

In [41]:
use std::fs::File;
use std::io;
use std::io::prelude::*;
use memmap::Mmap;

fn main() -> Result<(), io::Error> {
    let file = File::open("large_file.bin")?;
    let mmap = unsafe { Mmap::map(&file)? };

    // Access the memory-mapped data as a slice.
    let data = &mmap;

    // Perform read operations on 'data'.
    let max_bytes_to_print = 100;
    for i in 0..max_bytes_to_print {
        // Convert the byte to an ASCII code character and print it.
        print!("{} ", data[i] as char);
    }

    Ok(())
}

main()

Ok(())

### Network Programming and Device I/O

#### Sockets and Protocols

In [6]:
use std::net::TcpListener;
use std::io::{Read, Write};

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let listener = TcpListener::bind("0.0.0.0:8080")?;
    println!("Sensor server listening on 0.0.0.0:8080");

    for stream in listener.incoming() {
        let mut stream = stream?;
        let mut buffer = [0; 1024];
        stream.read(&mut buffer)?;

        // Process sensor data from the remote device
        let sensor_data = process_sensor_data(&buffer);

        // Perform device I/O operations with the sensor data
        perform_device_io(&sensor_data);

        // Respond to the remote device
        stream.write_all(b"Data received and processed.")?;
    }

    Ok(())
}

fn process_sensor_data(data: &[u8]) -> SensorData {
    // Process the received data and convert it into a structured format
    // In a real-world scenario, this could involve deserialization and validation
    // For simplicity, we assume direct conversion in this example
    SensorData::from_bytes(data)
}

fn perform_device_io(sensor_data: &SensorData) {
    // Perform device I/O operations using the sensor data
    // This could involve controlling actuators, making decisions, or storing data
    // In this example, we perform a hypothetical device I/O operation
    // by printing the sensor data
    println!("Received sensor data: {:?}", sensor_data);
}

#[derive(Debug)]
struct SensorData {
    // Define the structure for sensor data
}

impl SensorData {
    fn from_bytes(_data: &[u8]) -> SensorData {
        // Convert the received bytes into a SensorData instance
        // In a real-world application, this might involve deserialization
        // and validation of the data.
        // In this simplified example, we assume the bytes directly represent
        // the sensor data.
        SensorData {}
    }
}

main()

Sensor server listening on 0.0.0.0:8080
Received sensor data: SensorData
Received sensor data: SensorData
Received sensor data: SensorData
Received sensor data: SensorData
Received sensor data: SensorData


Error: Subprocess terminated with status: signal: 9 (SIGKILL)

#### Asynchronous Networking and Device I/O

In [3]:
:dep tokio = { version = "1.35.0", features = ["full"] }

In [4]:
use tokio::net::TcpListener;
use tokio::io::{AsyncWriteExt, AsyncReadExt};
use std::error::Error;
use std::net::SocketAddr;

#[tokio::main]
async fn main() -> Result<(), Box<dyn Error>> {
    let addr: SocketAddr = "0.0.0.0:8080".parse()?;
    let listener = TcpListener::bind(&addr).await?;
    println!("Sensor server listening on 0.0.0.0:8080");

    loop {
        let (mut socket, addr) = listener.accept().await?;
        println!("new client: {:?}", addr);
        tokio::spawn(async move {
            let mut buf = vec![0; 1024];
            if let Err(e) = socket.read(&mut buf).await {
                eprintln!("Error reading from socket: {}", e);
                return;
            }

            // Process sensor data from the remote device
            let sensor_data = process_sensor_data(&buf);

            // Perform device I/O operations with the sensor data
            perform_device_io(&sensor_data);

            // Respond to the remote device with a custom response
            let response = b"Data received and processed\n";
            if let Err(e) = socket.write_all(response).await {
                eprintln!("Error writing to socket: {}", e);
                return;
            }
            // Flush the output buffer to ensure the response is sent immediately
            if let Err(e) = socket.flush().await {
                eprintln!("Error flushing socket: {}", e);
            }
        });
    }
}

fn process_sensor_data(data: &[u8]) -> SensorData {
    // Process the received data and convert it into a structured format
    // In a real-world scenario, this could involve deserialization and validation
    // For simplicity, we assume direct conversion in this example
    SensorData::from_bytes(data)
}

fn perform_device_io(sensor_data: &SensorData) {
    // Perform device I/O operations using the sensor data
    // This could involve controlling actuators, making decisions, or storing data
    // In this example, we perform a hypothetical device I/O operation
    // by printing the sensor data
    println!("Received sensor data: {:?}", sensor_data);
}

#[derive(Debug)]
struct SensorData {
    // Define the structure for sensor data
}

impl SensorData {
    fn from_bytes(_data: &[u8]) -> SensorData {
        // Convert the received bytes into a SensorData instance
        // In a real-world application, this might involve deserialization
        // and validation of the data.
        // In this simplified example, we assume the bytes directly represent
        // the sensor data.
        SensorData {}
    }
}

main()

Sensor server listening on 0.0.0.0:8080
new client: 127.0.0.1:37118
new client: 127.0.0.1:37114
new client: 127.0.0.1:37116
Received sensor data: SensorData
Received sensor data: SensorData
new client: 127.0.0.1:37144
new client: 127.0.0.1:37142
Received sensor data: SensorData
Received sensor data: SensorData
Received sensor data: SensorData
new client: 127.0.0.1:43962
Received sensor data: SensorData
new client: 127.0.0.1:43970
Received sensor data: SensorData
new client: 127.0.0.1:43972
Received sensor data: SensorData
new client: 127.0.0.1:43982
Received sensor data: SensorData
new client: 127.0.0.1:43996
Received sensor data: SensorData
new client: 127.0.0.1:39376
new client: 127.0.0.1:39386
Received sensor data: SensorData
Received sensor data: SensorData
new client: 127.0.0.1:39390
Received sensor data: SensorData
new client: 127.0.0.1:39392
new client: 127.0.0.1:39404
Received sensor data: SensorData
Received sensor data: SensorData
new client: 127.0.0.1:38146
Received sensor d

Error: Subprocess terminated with status: signal: 9 (SIGKILL)

In [2]:
use std::process::{Command, Output, Stdio};

// A helper function to execute a shell command from a Rust script
fn execute_command(command: &str) -> Result<(), std::io::Error> {
    let status = Command::new("bash")
        .arg("-c")
        .arg(command)
        .stderr(Stdio::inherit())
        .status()?;

    if status.success() {
        Ok(())
    } else {
        Err(std::io::Error::from_raw_os_error(status.code().unwrap_or(1)))
    }
}

In [ ]:
let command = "cd asynchronous-networking && cargo run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.03s
     Running `target/debug/asynchronous-networking`


Sensor server listening on 0.0.0.0:8080
Received sensor data: SensorData
Received sensor data: SensorData
Received sensor data: SensorData
Received sensor data: SensorData
Received sensor data: SensorData


#### Networking Libraries and Device I/O


In [7]:
:dep tokio = {version = "1.35.0", featrues=["full"]}

In [8]:
:dep hyper = {version = "0.14.27", features = ["http1", "server", "tcp"]}

In [11]:
use hyper::service::{make_service_fn, service_fn};
use hyper::{Body, Request, Response, Server};
use std::convert::Infallible;
use std::net::SocketAddr;

async fn device_control(_req: Request<Body>) -> Result<Response<Body>, Infallible> {
    // Implement device control logic here
    // This function could process incoming requests to control devices

    Ok(Response::new(Body::from("Device control response")))
}

#[tokio::main]
async fn main() -> Result<(), Box<dyn std::error::Error>> {
    let make_svc = make_service_fn(|_conn| {
        let svc = service_fn(device_control);
        async move { Ok::<_, Infallible>(svc) }
    });

    let addr: SocketAddr = ([127, 0, 0, 1], 8080).into();
    let server = Server::bind(&addr).serve(make_svc);

    println!("Device control server listening on http://127.0.0.1:8080");

    server.await?;

    Ok(())
}

main()
// TODO: Report an evcxr bug?

Error: failed to resolve: could not find `main` in `tokio`

Error: `impl Future<Output = Result<(), Box<(dyn std::error::Error + 'static)>>>` doesn't implement `Debug`

In [ ]:
let command = "cd networking-libraries && cargo run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

   Compiling networking-libraries v0.1.0 (/home/mahmoud/Desktop/Rust Book/rust-lang-book/chapter-9/networking-libraries)
    Finished dev [unoptimized + debuginfo] target(s) in 2.29s
     Running `target/debug/networking-libraries`


Device control server listening on http://127.0.0.1:8080


### Parallelism, Concurrency, and Device I/O

In [12]:
use std::thread;
use std::sync::{Arc, Mutex};

fn main() {
    let sensor_data = Arc::new(Mutex::new(SensorData {}));
    let device = Arc::new(Mutex::new(Device::new()));

    let sensor_thread = thread::spawn({
        let sensor_data = Arc::clone(&sensor_data);
        move || {
            // Read sensor data and update shared data
            let mut sensor_data = sensor_data.lock().unwrap();
            sensor_data.read();
            println!("Sensor thread finished reading data.");
        }
    });

    let device_thread = thread::spawn({
        let sensor_data_clone = Arc::clone(&sensor_data);
        let device_clone = Arc::clone(&device);
        move || {
            // Control the device based on sensor data
            let sensor_data = sensor_data_clone.lock().unwrap();
            let mut device = device_clone.lock().unwrap();
            device.control(&sensor_data);
            println!("Device thread finished controlling the device.");
        }
    });

    sensor_thread.join().unwrap();
    device_thread.join().unwrap();
    println!("Main thread finished.");
}

struct SensorData {
    // Define the structure for sensor data
}

impl SensorData {
    fn read(&mut self) {
        // Simulate reading sensor data
        println!("Reading sensor data...");
    }
}

struct Device {
    // Define the structure for the device
}

impl Device {
    fn new() -> Device {
        // Initialize the device
        Device {}
    }

    fn control(&mut self, _sensor_data: &SensorData) {
        // Control the device based on sensor data
        println!("Controlling the device...");
    }
}

main()

Reading sensor data...
Sensor thread finished reading data.
Controlling the device...
Device thread finished controlling the device.
Main thread finished.


()

---
---